In [18]:
#import necessary packages
import spotipy
from spotipy import util
from spotipy import oauth2
from pprint import pprint
import random
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import json
import os
import csv

In [19]:
#generate token for spotify API
username = 'ernyjyqoecerasugt8dpmhbmo'
scope = 'playlist-modify-public'
my_client_id = '83c842b2f4d447b6b2a19df91847487d'
my_client_secret = '8bd6101237114d9eb851e2d3183affce'

spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

sp = spotipy.Spotify(auth=spotify_token)

In [20]:
#import Billboard CSV
ChartsDF = pd.read_csv("Resources/Hot_100BBCharts_time.csv")
ChartsDF.head()

,Date,Title,Artist,Peak_Position,Last_Position,Weeks_on_Chart,Rank_in_Chart,New_on_Chart
0,1958-08-06,Poor Little Fool,Ricky Nelson,1,1,2,1,False
1,1958-08-06,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,2,54,2,2,False
2,1958-08-06,Patricia,Perez Prado And His Orchestra,2,2,2,3,False
3,1958-08-06,Splish Splash,Bobby Darin,3,3,2,4,False
4,1958-08-06,When,Kalin Twins,5,5,2,5,False


In [21]:
#Only keep song Title, Artist, and peak chart position
ChartsDF_small = ChartsDF[["Title","Artist","Peak_Position"]]
ChartsDF_small["Song, Artist"] = ChartsDF_small["Title"]+", "+ChartsDF_small["Artist"]
ChartsDF_small.head()

C:\Users\codyb\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Title,Artist,Peak_Position,"Song, Artist"
0,Poor Little Fool,Ricky Nelson,1,"Poor Little Fool, Ricky Nelson"
1,Nel Blu Dipinto Di Blu (Volaré),Domenico Modugno,2,"Nel Blu Dipinto Di Blu (Volaré), Domenico Modugno"
2,Patricia,Perez Prado And His Orchestra,2,"Patricia, Perez Prado And His Orchestra"
3,Splish Splash,Bobby Darin,3,"Splish Splash, Bobby Darin"
4,When,Kalin Twins,5,"When, Kalin Twins"


In [22]:
#Group DF by Song, Artist
GroupedChartsDF = ChartsDF_small.groupby(['Song, Artist']).min()
GroupedChartsDF.head()

,Title,Artist,Peak_Position
"Song, Artist",,,
"""B"" Girls, Young And Restless","""B"" Girls",Young And Restless,54
"""Cherry Cherry"" from Hot August Night, Neil Diamond","""Cherry Cherry"" from Hot August Night",Neil Diamond,31
"""Having A Party"" Medley, The Ovations (Featuring Louis Williams)","""Having A Party"" Medley",The Ovations (Featuring Louis Williams),56
"""Joy"" Pt. I, Isaac Hayes","""Joy"" Pt. I",Isaac Hayes,30
"""Roots"" Medley, Quincy Jones","""Roots"" Medley",Quincy Jones,57


In [23]:
#Create column to store results of fuzzy match
GroupedChartsDF["Song ID"] = ""
GroupedChartsDF.head()

,Title,Artist,Peak_Position,Song ID
"Song, Artist",,,,
"""B"" Girls, Young And Restless","""B"" Girls",Young And Restless,54,
"""Cherry Cherry"" from Hot August Night, Neil Diamond","""Cherry Cherry"" from Hot August Night",Neil Diamond,31,
"""Having A Party"" Medley, The Ovations (Featuring Louis Williams)","""Having A Party"" Medley",The Ovations (Featuring Louis Williams),56,
"""Joy"" Pt. I, Isaac Hayes","""Joy"" Pt. I",Isaac Hayes,30,
"""Roots"" Medley, Quincy Jones","""Roots"" Medley",Quincy Jones,57,


In [ ]:
#Loop through songs and find best match from Spotify API
song_ids = []

attempts = 3000

for index, row in GroupedChartsDF.iterrows():
    
    if attempts == 3000:
        spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

        sp = spotipy.Spotify(auth=spotify_token)
        
        attempts = 0
        
        print("token reset")
    
    attempts += 1
    
    test_song = index

    results = sp.search(q=test_song, type='track', limit=10)

    search_match_score = []
    
    try:
    
        for x in range(len(results["tracks"]["items"])):
            song = results["tracks"]["items"][x]["name"]
            artist = results["tracks"]["items"][x]["artists"][0]["name"]
            song_id = results["tracks"]["items"][x]["id"]
            search_match_score.append(fuzz.ratio(test_song,song+","+artist))

        max_loc = 0
        max_val = 0
        for y in range(len(results["tracks"]["items"])):
            if max_val < search_match_score[y]:
                max_loc = y
                max_val = search_match_score[y]
        
        if max_val >= 75:
            song_ids.append(song_id)
        else:
            song_ids.append("")
    
    except:
        song_ids.append("")
        
GroupedChartsDF["Song ID"] = song_ids

retrying ...1secs
